In [1]:
import pandas as pd
%matplotlib inline

In [2]:
import os
os.chdir("G:\data")

In [3]:
hrtrain=pd.read_csv("train_LZdllcl.csv")

In [4]:
hrtest=pd.read_csv("test_2umaH9m.csv")

In [5]:
hrtrain.shape,hrtest.shape

((54808, 14), (23490, 13))

In [6]:
hr=pd.concat([hrtrain,hrtest],axis=0)

In [7]:
hr.head().columns
d_r_r=pd.get_dummies(hr[["department","recruitment_channel","region"]],drop_first=True)
hr.head()
def gen(i):
    if i=="m":
        return 1
    elif i=="f":
        return 0
    


In [8]:

hr["gender"]=hr["gender"].apply(gen)

In [9]:
d_r_r
hr.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted'],
      dtype='object')

In [10]:
ids=hrtest.iloc[:,0]
ids
hr.education.value_counts()
def ed(i):
    if i=="Below Secondary":
        return 0
    elif i=="Bachelor's":
        return 1
    elif i=="Master's & above":
        return 2
    
hr["education"]=hr["education"].apply(ed)

In [11]:
hrf=pd.concat([hr.drop(["employee_id","department","region","recruitment_channel"],axis=1),d_r_r],axis=1)

In [31]:
hrf.isnull().sum()
hrf.loc[(hrf["is_promoted"]==1)&(hrf["age"]==20)&(hrf["education"].isnull()),"education"]=0

In [62]:
hrf.loc[(hrf["is_promoted"]==1)&(hrf["age"]<22)&(hrf["education"].isnull()),"education"]=0

In [73]:
hrf.loc[(hrf["is_promoted"]==1)&(hrf["age"]<28)&(hrf["education"].isnull()),"education"]=1

In [96]:
hrf_tr=hrf.iloc[:54808,:]
X=hrf_tr.drop("is_promoted",axis=1)


In [97]:
pip install imutils

  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25860 sha256=58fe4163169dc269a34b374a43f9a46cbda607fc7e5abe102bd93591a63244ed
  Stored in directory: c:\users\hp\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils



In [98]:
!git clone https://github.com/analokmaus/kuma_utils.git
import sys
sys.path.append("kuma_utils/")

'git' is not recognized as an internal or external command,
operable program or batch file.


In [99]:
#hrf.isnull().sum()
#hrf.loc[(hrf["is_promoted"]==1)&(hrf["age"]>=28)&(hrf["length_of_service"]>=5)]  #["education"].value_counts()
from kuma_utils.preprocessing.imputer import LGBMImputer
lgb=LGBMImputer(n_iter=100,verbose=True)
X=lgb.fit_transform(X)

ModuleNotFoundError: No module named 'kuma_utils'

In [ ]:
hrf.isnull().sum()
hrf.loc[(hrf["is_promoted"]==1)&(hrf["age"]>=37)&(hrf["education"].isnull()),"education"]=2

In [ ]:
hrf["education"]=hrf["education"].fillna(1)

In [ ]:
hrf.loc[(hrf["is_promoted"]==1)&(hrf["previous_year_rating"].isnull()),"previous_year_rating"]=4

In [ ]:
hrf["previous_year_rating"]=hrf["previous_year_rating"].fillna(3)

In [ ]:
hrf_tr=hrf.iloc[:54808,:]
hrf_ts=hrf.iloc[54808:,:]
hrf_tr.shape,hrf_ts.shape

In [ ]:
X=hrf_tr.drop("is_promoted",axis=1)
y=hrf_tr["is_promoted"]

In [ ]:
#from sklearn.linear_model import LogisticRegression
#log=LogisticRegression()
from xgboost import XGBClassifier
xgb = XGBClassifier()
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score,classification_report
f1=[]
skf=StratifiedKFold(n_splits=5,random_state=None)
skf.get_n_splits(X,y)
for train_index,test_index in skf.split(X,y):
    print("Train:", train_index, "Validation:", test_index)
    x1_train,x1_test=X.iloc[train_index],X.iloc[test_index]
    y1_train,y1_test=y.iloc[train_index],y.iloc[test_index]
    xgb.fit(x1_train,y1_train)
    prediction=xgb.predict(x1_test)
    score=f1_score(y1_test,prediction)
    f1.append(score)

print(f1)
print(classification_report(y1_test,prediction))

In [ ]:
xt=hrf_ts.drop("is_promoted",axis=1)
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X,y)
pred1=xgb.predict(xt)

In [ ]:
submission=pd.DataFrame({'employee_id':hrtest.employee_id,
                        'is_promoted':pred1})

submission.to_csv("HR1.csv",index=False)

In [ ]:
#import xgboost as xgb
#pip install xgboost
!pip install xgboost

In [ ]:
hrtest.shape[0]

In [ ]:
hr=pd.concat([hrtrain,hrtest],axis=0)

In [ ]:
hr.head()
hr.shape

In [ ]:
hr.isnull().sum()

In [ ]:
pd.crosstab(hr["employee_id"],hr["education"])
hr.groupby("education")[["employee_id"]].mean()
hr[["previous_year_rating","employee_id"]].corr()
hr.drop("employee_id",axis=1,inplace=True)

In [ ]:
import matplotlib.pyplot as plt
#pd.crosstab(hr["department"],hr["education"]).plot(kind="bar")
# Sales and marketing has highest number of bachelor's

In [ ]:
plt.rcParams["figure.figsize"]=[15, 8]
#pd.crosstab(hr["region"],hr["education"]).plot(kind="bar")
#Region 2,22,7 have highest number of bachelor's and master's

In [ ]:
#pd.crosstab(hr["gender"],hr["education"]).plot(kind="bar")
# Most of the Bachelor's and Master's are males

In [ ]:
#pd.crosstab(hr["recruitment_channel"],hr["education"]).plot(kind="bar")
#Most bachelor's and master's were recruited through channels other than referals.

In [ ]:
#pd.crosstab(hr["no_of_trainings"],hr["education"]).plot(kind="bar")
hr.no_of_trainings.value_counts()
#Most of the people required upto 3 trainings.

In [ ]:
#hr.groupby("education")[["age"]].mean().plot(kind="bar")
#People with Master's and above degrees have highest mean age.

In [ ]:
#pd.crosstab(hr["previous_year_rating"],hr["education"]).plot(kind="bar")
#Most bachelor's and masters have atleast 3 previous year ratigs

In [ ]:
#hr.groupby("education")[["length_of_service"]].mean().plot(kind="bar")
#bachelor's and masters have highest length of service

In [ ]:
#pd.crosstab(hr["KPIs_met >80%"],hr["education"]).plot(kind="bar")
#Few people met KPIs

In [ ]:
#pd.crosstab(hr["awards_won?"],hr["education"]).plot(kind="bar")
#Most people didnt win award

In [ ]:
#hr.groupby("education")[["avg_training_score"]].mean().plot(kind="bar")
#No trend seen

In [ ]:
#pd.crosstab(hr["is_promoted"],hr["education"]).plot(kind="bar")
#Few got promoted

In [ ]:
hr.isnull().sum()

In [ ]:
#hr.loc[(hr["is_promoted"]==1)&(hr["awards_won?"]==1)&(hr["KPIs_met >80%"]==1)&(hr["previous_year_rating"].isnull())][["previous_year_rating"]].fillna(5,inplace=True)
    



    

In [ ]:
#pd.crosstab(hr["previous_year_rating"],hr["no_of_trainings"]).plot(kind="bar")
import numpy as np
hr.loc[(hr["previous_year_rating"]==5)]["avg_training_score"].mean()


In [ ]:
hr.education.value_counts()

In [ ]:
hr.previous_year_rating.value_counts()

In [ ]:
hr.isnull().sum()

In [ ]:
hr["education"].fillna("Bachelor's", inplace=True)

In [ ]:
hr["education"].isnull().sum()

In [ ]:
hr.iloc[:,:]

In [ ]:
hr["previous_year_rating"].value_counts()
hr.columns

In [ ]:
cat=['department','region','education','gender','recruitment_channel','no_of_trainings','KPIs_met >80%','awards_won?']
num=['age','length_of_service','avg_training_score']
columns=['department', 'region', 'education', 'gender', 'recruitment_channel',
       'no_of_trainings', 'age', 'length_of_service',
       'KPIs_met >80%', 'awards_won?', 'avg_training_score']

In [ ]:
#plt.rcParams["figure.figsize"]=[15,8]
#for i in cat:
    #pd.crosstab(hr["previous_year_rating"],hr[i]).plot(kind="bar")
# Strong likelihood that if person belongs to operations,procurement or tech dept then his rating is atleast 3.
# People from region 7 have rating 3 or more.

In [ ]:
#for j in num:
   # hr.groupby("previous_year_rating")[[j]].mean().plot(kind="bar")

In [ ]:
hr["previous_year_rating"].fillna(4.0,inplace=True)

In [ ]:
hr["is_promoted"].value_counts()

In [ ]:
hr.isnull().sum()

In [ ]:
hr.head()

In [ ]:
hr[['age', 'length_of_service', 'avg_training_score']]

In [ ]:
cat
hr.head()

In [ ]:
corecat=['department','region','education','gender','recruitment_channel']
print((hr.loc[(hr["gender"]=="f")&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["gender"]=="f"].shape[0])
(hr.loc[(hr["gender"]=="m")&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["gender"]=="m"].shape[0]
# So we will give more weight to women as they have better chance of promotion
    



In [ ]:
print((hr.loc[(hr["education"]=="Bachelor's")&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["education"]=="Bachelor's"].shape[0])
print((hr.loc[(hr["education"]=="Master's & above")&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["education"]=="Master's & above"].shape[0])
print((hr.loc[(hr["education"]=="Below Secondary")&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["education"]=="Below Secondary"].shape[0])
# We see that Masters and above have higher chances of promotion than the rest.

In [ ]:
print((hr.loc[(hr["recruitment_channel"]=="other")&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["recruitment_channel"]=="other"].shape[0])
print((hr.loc[(hr["recruitment_channel"]=="sourcing")&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["recruitment_channel"]=="sourcing"].shape[0])
print((hr.loc[(hr["recruitment_channel"]=="referred")&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["recruitment_channel"]=="referred"].shape[0])
#Refered>sourcing>other weights in promotion

In [ ]:
hr["region"].value_counts().index
reg=['region_2', 'region_22', 'region_7', 'region_15', 'region_13',
       'region_26', 'region_31', 'region_4', 'region_27', 'region_16',
       'region_28', 'region_11', 'region_23', 'region_29', 'region_32',
       'region_19', 'region_14', 'region_20', 'region_17', 'region_25',
       'region_5', 'region_6', 'region_30', 'region_8', 'region_10',
       'region_1', 'region_24', 'region_12', 'region_9', 'region_21',
       'region_3', 'region_34', 'region_33', 'region_18']

In [ ]:
dic={}
for i in reg:
    dic.update({i:(round((hr.loc[(hr["region"]==i)&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["region"]==i].shape[0],3))})
print(dic)
hr["region"]=hr["region"].map(dic)
    

In [ ]:
hr["region"].value_counts()
    

        

        

In [ ]:
hr.isnull().sum()

In [ ]:
hr.insert(loc = 0,column = 'new_ftr',value = hr["KPIs_met >80%"]+hr["awards_won?"])

In [ ]:
hr["new_ftr"].fillna(0,inplace=True)

In [ ]:
hr.columns
hr.drop(["KPIs_met >80%","awards_won?"],axis=1,inplace=True)

In [ ]:
hr["department"].value_counts().index
dept=['Sales & Marketing', 'Operations', 'Procurement', 'Technology',
       'Analytics', 'Finance', 'HR', 'Legal', 'R&D']

In [ ]:
dics={}
for i in dept:
    dics.update({i:round((hr.loc[(hr["department"]==i)&(hr["is_promoted"]==1)].shape[0])/hr.loc[hr["department"]==i].shape[0],3)})
hr["department"]=hr["department"].map(dics)

    

In [ ]:
hr["department"]  

        

In [ ]:
hr.education.value_counts()

In [ ]:
hr["gender"].replace({"f":2,"m":1},inplace=True)

In [ ]:
def edu(i):
    if i=="Master's & above":
        return 2
    else:
        return 1
hr["education"]=hr["education"].apply(edu)

In [ ]:
hr["recruitment_channel"].value_counts()
def rec(i):
    if i=="other":
        return 1
    elif i=="sourcing":
        return 2
    else:
        return 3
    
hr["recruitment_channel"]=hr["recruitment_channel"].apply(rec)
    

In [ ]:
hr.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
hr["age"]=scaler.fit_transform(hr[["age"]])
hr["length_of_service"]=scaler.fit_transform(hr[["length_of_service"]])
hr["avg_training_score"]=scaler.fit_transform(hr[["avg_training_score"]])
hr["previous_year_rating"]=scaler.fit_transform(hr[["previous_year_rating"]])

In [ ]:
hr.head()

In [ ]:
hr["is_promoted"].fillna(0,inplace=True)

In [ ]:
hr.isnull().sum()
hr.is_promoted.value_counts()

In [ ]:
hr.isnull().sum()

In [ ]:
hr["is_promoted"]=hr["is_promoted"].astype(int)

In [ ]:
hr.info()

In [ ]:

x=hr.iloc[:,:-1]
y=hr.iloc[:,-1]

In [ ]:
pd.DataFrame(y)

In [ ]:
hr.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.3)



In [ ]:
xtest.shape[0]

In [ ]:
ytest.isnull().sum()

In [ ]:
from lightgbm import LGBMClassifier
lgb=LGBMClassifier()

In [ ]:
lgb.fit(xtrain,ytrain)

In [ ]:
pred_y=lgb.predict(xtest)

In [ ]:
from sklearn.metrics import f1_score
f1_score(ytest, pred_y)

In [ ]:
from pandas import DataFrame,Series
z=pd.concat([ids,pd.DataFrame(pred_y)],axis=1)
z.columns=["employee_id","is_promoted"]
z.to_csv("AV1.csv",index=False)





In [ ]:
#prediction1 = pd.DataFrame([hrtest.iloc[:,[0]],pred_y], columns=['emp_id','predictions']).to_csv('prediction.csv')

from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

x, y = make_classification(n_samples=78298, n_features=12, 
                           n_classes=2, 
                           n_clusters_per_class=1)

xtrain, xtest, ytrain, ytest=train_test_split(x, y, test_size=0.15)

lsvc = LinearSVC()
print(lsvc)

lsvc.fit(xtrain, ytrain)
score = lsvc.score(xtrain, ytrain)
print("Score: ", score)

cv_scores = cross_val_score(lsvc, xtrain, ytrain, cv=10)
print("CV average score: %.2f" % cv_scores.mean())

ypred = lsvc.predict(xtest)

cm = confusion_matrix(ytest, ypred)
print(cm)

cr = classification_report(ytest, ypred)
print(cr) 



